
# WGS84 to HEALPix Conversion Using Python

In this notebook, we will demonstrate how to convert geographic coordinates (latitude, longitude) based on the WGS84 reference system into HEALPix pixel indices. This involves transforming coordinates into a geocentric Cartesian system and subsequently into spherical coordinates that are compatible with HEALPix.

---

## What is WGS84?

WGS84 (World Geodetic System 1984) is a global reference system for geographic coordinates (latitude, longitude, and altitude). It is widely used for mapping, GPS, and geospatial data applications.

For more details, you can refer to the [WGS84 definition](https://en.wikipedia.org/wiki/World_Geodetic_System).

---

## What is HEALPix?
For more information on HEALPix, visit the [HEALPix Documentation](https://healpix.sourceforge.io/).


---

## Setup

To run this notebook, you need to install the following libraries:
1. **Pyproj**: For coordinate transformations.
2. **Healpy**: For working with HEALPix, a hierarchical spatial indexing system.
3. **XDGGS**: For working with HEALPix together with Xarray to handle simpler data manipulation.

Additionally, if you are using XDGGS for spatial data applications, you will need to install the XDGGS library along with Lonboard. You can find the repository here: [XDGGS GitHub Repository](https://github.com/xdggs/xdggs).

To install the necessary dependencies, use the following command:

```bash
pip install pyproj healpy xdggs



In [2]:
# Import necessary libraries
import pyproj
import numpy as np
import healpy as hp  # HEALPix library


## Initialize Coordinate Reference Systems (CRS)

Here, we initialize two CRS:
1. **WGS84**: A geographic CRS (latitude, longitude).
2. **Geocentric Cartesian**: A 3D Cartesian system for transformations involving Earth-centered coordinates.

In [3]:
# Initialize CRS (Coordinate Reference Systems)
wgs84 = pyproj.CRS("EPSG:4326")  # WGS84 (geographic latitude, longitude)
geocentric = pyproj.CRS("EPSG:4978")  # Geocentric Cartesian coordinate system (based on WGS84)

## Create a Transformer for WGS84 to Geocentric Conversion

Using `pyproj`, we create a transformer that converts geographic coordinates (latitude, longitude) into a geocentric Cartesian system. This is necessary for further spherical transformations.

In [4]:
# Create a transformer to convert WGS84 -> geocentric
transformer = pyproj.Transformer.from_crs(wgs84, geocentric, always_xy=True)

## Define Function: WGS84 to Spherical Coordinates

This function converts WGS84 geographic coordinates into HEALPix-compatible spherical coordinates (theta, phi). These coordinates are essential for identifying the correct HEALPix pixel index.

In [5]:
# Define a function to convert WGS84 (lat, lon) to spherical coordinates (HEALPix compatible)
def wgs84_to_spherical(lat, lon):
    """
    Converts geographic coordinates (latitude, longitude) to HEALPix-compatible spherical coordinates.
    
    Parameters:
        lat (float): Latitude in degrees.
        lon (float): Longitude in degrees.
        
    Returns:
        tuple: Spherical coordinates (theta, phi) in radians.
    """
    # Convert to geocentric Cartesian coordinates
    x, y, z = transformer.transform(lon, lat, 0)  # Altitude = 0 (on the ellipsoid)

    # Convert to spherical angles
    r = np.sqrt(x**2 + y**2 + z**2)  # Radius
    theta = np.arccos(z / r)  # Co-latitude in radians
    phi = np.arctan2(y, x)  # Longitude in radians
    return theta, phi

## Define Function: HEALPix Pixel Index

This function calculates the HEALPix pixel index for a given set of geographic coordinates using the resolution parameter `nside`.

In [6]:
# Define a function to calculate the HEALPix pixel index
def ang2pix_wgs84(lat, lon, nside):
    """
    Converts WGS84 coordinates to a HEALPix pixel index.
    
    Parameters:
        lat (float): Latitude in degrees.
        lon (float): Longitude in degrees.
        nside (int): HEALPix resolution parameter.
        
    Returns:
        int: Pixel index corresponding to the given coordinates.
    """
    # Convert WGS84 to spherical coordinates
    theta, phi = wgs84_to_spherical(lat, lon)
    
    # Return the HEALPix pixel index
    return hp.ang2pix(nside, theta, phi)

## Example Usage

Finally, we use the functions defined above to calculate the HEALPix pixel index for a specific location, in this case, Paris.

In [7]:
# Example usage
nside = 64  # HEALPix resolution
lat, lon = 48.8566, 2.3522  # Latitude and longitude of Paris

# Get the HEALPix pixel index for the given coordinates
pix = ang2pix_wgs84(lat, lon, nside)

# Print the result
print(f"Pixel index: {pix}")

Pixel index: 5941


## Conclusion

This notebook demonstrates the process of converting WGS84 coordinates to HEALPix indices. The workflow involves:
1. Transforming WGS84 coordinates into a geocentric Cartesian system.
2. Converting the Cartesian coordinates into spherical coordinates.
3. Calculating the HEALPix pixel index using the spherical coordinates.


---

## Next Step

We will show:
1. The transformation does not loose the benefit specific of HealPIx, such as spherical harmonics properties.  
2. Comparision with rHealPix for err geolocalisation.  

---